# 02. Registration to MNI Space

This notebook registers subject 7T data to MNI standard space using ANTs SyN.

## Why Registration Matters

The LC atlas is defined in MNI space. To extract signal from the LC region, we must:
1. Register subject T1w → MNI template
2. Apply the same transform to all contrast maps (R1, R2*, QSM)

Brainstem registration is challenging due to:
- Small structure sizes
- Lower contrast in deep brain regions
- Susceptibility artifacts near air-tissue interfaces

We use ANTs SyN (Symmetric Normalization) which is well-suited for subcortical alignment.

In [ ]:
import sys
sys.path.insert(0, '..')

import ants
import nibabel as nib

from src.config import DEFAULT_CONFIG
from src.io import list_subjects, ensure_output_dirs, load_nifti
from src.preprocessing import register_subject, get_mni_template
from src.visualization import plot_registration_qc

print(f"ANTsPy version: {ants.__version__}")

In [ ]:
# Configuration
config = DEFAULT_CONFIG
ensure_output_dirs(config)

print(f"Registration type: {config.registration_type}")
print(f"SyN metric: {config.syn_metric}")
print(f"Iterations: {config.reg_iterations}")

## 1. Load MNI Template

We use the MNI152 ICBM 2009c template for registration.

In [ ]:
# Load MNI template
fixed = get_mni_template()
print(f"Template shape: {fixed.shape}")
print(f"Template spacing: {fixed.spacing}")

## 2. Find Available Subjects

In [ ]:
subjects = list_subjects(config.data_dir)
print(f"Found {len(subjects)} subjects")

if not subjects:
    raise ValueError(
        "No subjects found. Please download AHEAD data from: "
        "https://doi.org/10.21942/uva.10007840"
    )

## 3. Register Single Subject (Development)

First, we develop the registration on one subject before batch processing.

In [ ]:
# Process first subject as test
test_sub = subjects[0]
print(f"Processing {test_sub}...")

results = register_subject(test_sub, fixed, config)

if results['success']:
    print(f"\nRegistration successful!")
    print(f"Outputs:")
    for key, value in results.items():
        if key.endswith('_MNI'):
            print(f"  {key}: {value}")
else:
    print(f"\nRegistration failed.")

## 4. Visual QC

Check registration quality by overlaying warped T1w on template at brainstem levels.

In [ ]:
if results['success']:
    # Load warped image
    warped_t1w = load_nifti(results['T1w_MNI'])
    
    # Generate QC figure
    qc_path = config.figures_dir / f"{test_sub}_registration_qc.png"
    
    plot_registration_qc(
        warped_img=warped_t1w,
        subject_id=test_sub,
        output_path=qc_path,
    )
    
    print(f"QC figure saved: {qc_path}")

## 5. Batch Processing (Optional)

Process all subjects. This will take significant time.

In [ ]:
# Uncomment to run batch processing
# WARNING: This will take a long time (10-30 min per subject)

# import json
# 
# all_results = []
# 
# for i, sub_id in enumerate(subjects):
#     print(f"\n[{i+1}/{len(subjects)}] Processing {sub_id}...")
#     
#     # Skip if already processed
#     sub_out_dir = config.output_dir / sub_id
#     if (sub_out_dir / f'{sub_id}_T1w_MNI.nii.gz').exists():
#         print(f"  Already processed, skipping.")
#         continue
#     
#     try:
#         results = register_subject(sub_id, fixed, config)
#         all_results.append(results)
#     except FileNotFoundError as e:
#         print(f"  Error: {e}")
#         all_results.append({'subject_id': sub_id, 'success': False, 'error': str(e)})
#     
#     # Save progress
#     log_path = config.output_dir / 'registration_log.json'
#     with open(log_path, 'w') as f:
#         json.dump(all_results, f, indent=2, default=str)
# 
# print(f"\nProcessed {len(all_results)} subjects.")
# print(f"Successful: {sum(r.get('success', False) for r in all_results)}")

## 6. Summary

This notebook:
1. Registered T1w images to MNI152 space using ANTs SyN
2. Applied the same transforms to R1, R2*, and QSM maps
3. Generated QC figures for visual inspection

**Next**: Apply LC atlas to extract signal (Notebook 03)